# Real State Scrapy

Scrapy the real state website and retrieve house listing of a target price and retrieve the info and save to database.

### Import Dependencies

In [1]:
import os
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd
import time
import numpy as np
import pprint
import datetime

# Import DB user and password
from api_keys import positionstack_key
from api_keys import opencagedata_API
from api_keys import DATABASE_URL

### Setup Splinter (For Mac)

Check Splinter documentation for info on how to find elements:
https://splinter.readthedocs.io/en/latest/finding.html

In [2]:
# identify location of chromedriver and store it as a variable
driverPath = !which chromedriver

# Setup configuration variables to enable Splinter to interact with browser
executable_path = {'executable_path': driverPath[0]}
browser = Browser('chrome', **executable_path, headless=False)

### Scraping

In [3]:
# URL of page to be scraped
url_realtor = "https://www.realtor.com/realestateandhomes-search/Houston_TX/type-single-family-home/price-"
link_details = "https://www.realtor.com"
min_price = '200000'
max_price = '300000'
sort_by = '/sby-2' # Highest to lowest price
sort_by = '/sby-6' # Newest listings
page_number = 4

query_url = f"{url_realtor}{min_price}-{max_price}{sort_by}/pg-{page_number}"
print(query_url)

https://www.realtor.com/realestateandhomes-search/Houston_TX/type-single-family-home/price-200000-300000/sby-6/pg-4


### BeautifulSoup

In [4]:
# Scrap with BeautifulSoup. However it does 
# page = requests.get(query_url)
# soup = BeautifulSoup(page.content, 'html.parser')

### Splinter

In [5]:
# Use the browser to visit the url
browser.visit(query_url)

In [6]:
# Wait for x seconds for error purpouses
time.sleep(5)

In [7]:
# Return the rendered page by the browser
html_realtor = browser.html

In [8]:
# Use beatifulsoup to scrap the page rendered by the browser
soup = BeautifulSoup(html_realtor, 'html.parser')

In [9]:
"""
Find all lis in order to find each house item.
Then look for the class = "photo-wrap" to retrieve the unique id
for each listing. Hover the mouse over the house to load the picture and
finally get the picture link to save in the database
"""
house_ads = soup.find_all('li')
for kk in range(len(house_ads)):
    if house_ads[kk].find_all('div', class_= "photo-wrap"):
        div_tag = house_ads[kk].find('div', class_= "photo-wrap")
        browser.find_by_id(div_tag['id']).mouse_over()
        time.sleep(1)

In [10]:
# Return the rendered page by the browser after loading all photos
html_realtor = browser.html

In [11]:
# Use beatifulsoup to scrap the page rendered by the browser
soup = BeautifulSoup(html_realtor, 'html.parser')

In [12]:
# Search for the div where the title is located
results = soup.find_all('div', class_="card-box")
# print(results[0].prettify())
print(f"Total results: {len(results)}")

Total results: 48


In [13]:
# Find beds, baths, sqft and lot
def find_features(house_feat_temp):
    list_features = []
    for jj in range(len(house_feat_temp)):
        list_features.append(house_feat_temp[jj].text)

    try:
        index_pos = list_features.index("bed")
    except ValueError:
        bed = 1.0
    else:
        bed = float(list_features[index_pos-1])

    try:
        index_pos = list_features.index("bath")
    except ValueError:
        bath = 1.0
    else:
        bath = float(list_features[index_pos-1].replace("+",""))

    try:
        index_pos = list_features.index("sqft")
    except ValueError:
        sqft = np.nan
    else:
        sqft = float(list_features[index_pos-1].replace(",",""))

    try:
        index_pos = list_features.index("sqft lot")
    except ValueError:
        try:
            index_pos = list_features.index("acre lot")
        except ValueError:
            lot = np.nan
        else:
            lot = round(float(list_features[index_pos-1])*43560)    
    else:
        lot = float(list_features[index_pos-1].replace(',',''))

#     print(f"Beds: {bed}, Bath: {bath}, Sqft: {sqft}, Lot: {lot}")
    
    return {"Beds": bed, "Bath": bath, "Sqft": sqft, "Lot": lot}

find_features(results[6].find('ul').find_all('span'))

{'Beds': 3.0, 'Bath': 2.5, 'Sqft': 1902.0, 'Lot': nan}

In [14]:
# Print results and save to a dictionary
n = 0
realstate_list = []
for result in results:
#     Clear the variables to not store repeated info
    house_price = ''
    address = ''
    link_page = ''
    photo_url = ''
                
    n = n + 1
    print('-----------------------------------')
    print('')
    print(f'Result: {n} of {len(results)}')
    if not result.find('div', class_="ads"):
#         print(f'Result: {n} of {len(results)}')
        price_div = result.find('div', class_="price")
        house_price = float(price_div.find('span').text.split('$')[-1].replace(",",""))
        link_page = result.find('a')['href']
        img_label = result.find('img')
        address = img_label['alt']
        
        
#         house_features = result.find('ul').text
#         list_house_features = house_features.replace("bed",";").replace("bath",";").replace("lot",";").replace("sqft",";").split(";")
#         bed = float(list_house_features[0])
#         bath = float(list_house_features[1])
#         sqft = int(list_house_features[2].replace(",",""))
#         lot = list_house_features[3]

#         # Convert acre to sqft
#         if (lot.find('acre') != -1): 
#             lot = (round(float(lot.replace('acre',''))*43560))
#         else:
#             lot = int(lot.replace(",",""))

        

        features = find_features(result.find('ul').find_all('span'))
        
        bed = features['Beds']
        bath = features['Bath']
        sqft = features['Sqft']
        lot = features['Lot']
            
        print(f"Price: ${house_price} | Beds: {bed}, Bath: {bath}, Sqft: {sqft}, Lot: {lot}")
        
        print(f"Address: {address}")
        
        try:
            price_reduced = result.find('span', class_="price-reduced-amount")
            price_now = price_reduced.text
            print(f"Price reduced: {price_now}")
        except:
            pass
        print(f"Link: {link_details}{link_page}")
#         photo_url = img_label['srcset'].split(',')[1].split(' ')[1]
        
#         try:
#             photo_url = img_label['srcset'].split(',')[1].split(' ')[1]
#             print(f"Photo link: {photo_url}")
#         except KeyError:
#             print(f"Photo not available")
            
        separator = '+'
        address_google = separator.join(address.split(" "))
        url_google_maps = f"https://www.google.com/maps/place/{address_google}"
        print(f"Google Maps: {url_google_maps}")
    
        list_images = ['', '']
        house_pictures_tag = result.find_all('picture')
        for uu in range(len(house_pictures_tag)):
            list_images[uu] = house_pictures_tag[uu].find('img')['srcset'].split(',')[1].split(' ')[1]
            print(f"Image_{uu}: {list_images[uu]}")
        
        # Save results to a dictionary
        realstate_list.append(
            {
                "Price": house_price,
                "Address": address,
                "Beds": bed,
                "Baths": bath,
                "Sqft": sqft,
                "Lot": lot,
                "Image_1": list_images[0],
                "Image_2": list_images[1],
                "Link": str(link_details+link_page),
                "Google Maps": url_google_maps
            }
        )

    
    else:
        print('Data not available or Ads')

    print('')
    
    

-----------------------------------

Result: 1 of 48
Price: $280000.0 | Beds: 3.0, Bath: 2.5, Sqft: 2458.0, Lot: 9120.0
Address: 3810 Woodbridge Dr, Kingwood, TX 77339
Link: https://www.realtor.com/realestateandhomes-detail/3810-Woodbridge-Dr_Kingwood_TX_77339_M88961-28456
Google Maps: https://www.google.com/maps/place/3810+Woodbridge+Dr,+Kingwood,+TX+77339
Image_0: https://ap.rdcpix.com/7c9d7cac1b315b6ed250467bf1a20f8el-m1376079040od-w480_h360_x2.jpg
Image_1: https://ap.rdcpix.com/7c9d7cac1b315b6ed250467bf1a20f8el-m108964704od-w480_h360_x2.jpg

-----------------------------------

Result: 2 of 48
Price: $215000.0 | Beds: 4.0, Bath: 2.0, Sqft: 1700.0, Lot: 6600.0
Address: 711 Woodbine St, Houston, TX 77017
Link: https://www.realtor.com/realestateandhomes-detail/711-Woodbine-St_Houston_TX_77017_M86030-09347
Google Maps: https://www.google.com/maps/place/711+Woodbine+St,+Houston,+TX+77017
Image_0: https://ap.rdcpix.com/f796993f0705dce50e2cff547253b05al-m3516385302od-w480_h360_x2.jpg
Imag

Result: 47 of 48
Price: $210000.0 | Beds: 3.0, Bath: 2.5, Sqft: 1982.0, Lot: 9450.0
Address: 18703 Sun Lodge Ct, Houston, TX 77073
Link: https://www.realtor.com/realestateandhomes-detail/18703-Sun-Lodge-Ct_Houston_TX_77073_M74742-83226
Google Maps: https://www.google.com/maps/place/18703+Sun+Lodge+Ct,+Houston,+TX+77073
Image_0: https://ap.rdcpix.com/5f37fc08274dc13fd5052392c7adac39l-m96138853od-w480_h360_x2.jpg
Image_1: https://ap.rdcpix.com/5f37fc08274dc13fd5052392c7adac39l-m190009968od-w480_h360_x2.jpg

-----------------------------------

Result: 48 of 48
Data not available or Ads



In [15]:
# When you’ve finished testing, close your browser using browser.quit:
browser.quit()

<hr style="border: 3px solid red;">

### Data Cleaning

In [16]:
# Save the data to a dataframe
listing_df = pd.DataFrame(realstate_list)
# listing_df.to_csv(os.path.join('Database','ScrapedData.csv'))
listing_df.head(2)

,Price,Address,Beds,Baths,Sqft,Lot,Image_1,Image_2,Link,Google Maps
0,280000.0,"3810 Woodbridge Dr, Kingwood, TX 77339",3.0,2.5,2458.0,9120.0,https://ap.rdcpix.com/7c9d7cac1b315b6ed250467b...,https://ap.rdcpix.com/7c9d7cac1b315b6ed250467b...,https://www.realtor.com/realestateandhomes-det...,https://www.google.com/maps/place/3810+Woodbri...
1,215000.0,"711 Woodbine St, Houston, TX 77017",4.0,2.0,1700.0,6600.0,https://ap.rdcpix.com/f796993f0705dce50e2cff54...,https://ap.rdcpix.com/f796993f0705dce50e2cff54...,https://www.realtor.com/realestateandhomes-det...,https://www.google.com/maps/place/711+Woodbine...


In [17]:
# Dataframe to dictionary
# listing_df.to_dict(orient="records")

<hr style="border: 3px solid red;">

## Save Scraped House Data to Database

Info about primary key auto increment even with roll back: https://stackoverflow.com/questions/449346/mysql-auto-increment-does-not-rollback

In [18]:
 # Imports the method used to connect to DBs
from sqlalchemy import create_engine
from sqlalchemy import exc
from sqlalchemy import update

# Imports the methods needed to abstract python classes into database tables
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# function to establish a session with a connected database
from sqlalchemy.orm import Session

# database compliant datatypes
from sqlalchemy import Column, Integer, String, Float, ForeignKey, DateTime

# from config import db_user, dialect

In [19]:
# Create database connection
engine = create_engine(DATABASE_URL) 

In [20]:
# Create class to frame each real state instance
class RealState(Base):
    __tablename__ = 'realstatelisting'

    house_id = Column(Integer, primary_key=True)
    address = Column(String(300), unique=True, nullable=False)
    price = Column(Float, nullable=False)
    bed = Column(Float, nullable=True)
    bath = Column(Float, nullable=True)
    sqft = Column(Float, nullable=True)
    lot = Column(Float, nullable=True)
    latitude = Column(Float, nullable=True)
    longitude = Column(Float, nullable=True)
    house_link = Column(String(300), nullable=True)
    image_1 = Column(String(300), nullable=True)
    image_2 = Column(String(300), nullable=True)
    map_link = Column(String(300), nullable=True)
    google_map = Column(String(300), nullable=True)
    created_date = Column(DateTime, default=datetime.datetime.utcnow)

    def __repr__(self):
        return '<Listing %r>' % (self.address)

In [21]:
# Create class to frame each real state instance
class UserSelection(Base):
    __tablename__ = 'userselection'

    userselection_id = Column(Integer, primary_key=True)
    username = Column(String(300))
    useremail = Column(String(300))
    house_id = Column(Integer, ForeignKey('realstatelisting.house_id'))
    user_choice = Column(String(300))
    created_date = Column(DateTime, default=datetime.datetime.utcnow)
    
    def __repr__(self):
        return '<Listing %r>' % (self.userselection_id)

In [22]:
# Create all of the tables in our database based on the classes we've associated with our declarative base.
Base.metadata.create_all(engine)

In [23]:
# Create a Session object to connect to DB
session = Session(bind=engine)

In [24]:
""" Verify if the items are in the database to avoid roll back"""
new_entries_index = []
for nn in range(len(listing_df)):
    house_item = listing_df.iloc[nn]
    query_results = session.query(RealState).filter(RealState.address == house_item.Address).all()
    if query_results:
        pass
    else:
        new_entries_index.append(nn)

new_houses_df = listing_df.iloc[new_entries_index]
new_houses_df = new_houses_df.drop_duplicates()
print(f"--> {len(new_houses_df)} records to be added.")

--> 41 records to be added.


In [25]:
ll = 0
for nn in range(len(new_houses_df)):
#     print('-'*25)
#     print(f"{nn+1} of {len(listing_df)}")
    house_item = new_houses_df.iloc[nn]

    new_house = RealState(
        address = house_item.Address,
        price = house_item.Price,
        bed = house_item.Beds,
        bath = house_item.Baths,
        sqft = house_item.Sqft,
        lot = house_item.Lot,
        image_1 = house_item['Image_1'],
        image_2 = house_item['Image_2'],
        house_link = house_item.Link,
        google_map = house_item['Google Maps']
        )
    
    try:
        session.add(new_house)
        session.commit()
        ll = ll + 1
    except exc.IntegrityError:
        session.rollback()
        print(f'Roll back: {new_house}')

print(f"--> Total recordes added to database: {ll}.")

--> Total recordes added to database: 41.


<hr style="border: 3px solid red;">

### Get Coordinates Using Google Maps

In [26]:
""" Get house coordinates using Splinter to scrap Google Maps """
def find_coordinates(url):
    browser = Browser('chrome', **executable_path, headless=False)
    browser.visit(url)
    time.sleep(5)
    current_url = browser.url
    browser.quit()

    try:
        latitude = float(current_url.split("!")[-2].split("d")[1])
        longitude = float(current_url.split("!")[-1].split("d")[1])
    except IndexError:
        url_string_list = current_url.split("/")
        for jj in range(len(url_string_list)):
            try:
                index_coordinates = url_string_list[jj].index("@")
            except ValueError:
                pass
            else:
                latitude = float(url_string_list[jj][1:-1].split(",")[0])
                longitude = float(url_string_list[jj][1:-1].split(",")[1])
        
    map_link = f"https://www.openstreetmap.org/?mlat={latitude}&mlon={longitude}#map=15/{latitude}/{longitude}"

    return {"latitude":latitude, "longitude":longitude, "map_link": map_link}

In [27]:
""" Query houses and update entry with coordinates and map link """
def update_house_coordinates(HouseID):
    house = session.query(RealState).filter(RealState.house_id == HouseID)
    coordinates = find_coordinates(house[0].google_map)
    stmt = update(RealState).where(RealState.house_id == HouseID).values(coordinates).\
        execution_options(synchronize_session="fetch")

    session.execute(stmt)
    session.commit()


In [28]:
# Update coordinates for all entries of the database that doesn't have coordinates
house_list = session.query(RealState).filter(RealState.latitude == None).order_by(RealState.house_id).all()
nn = 0
for house in house_list:
    update_house_coordinates(house.house_id)
    nn = nn + 1

print(f"{nn} records updated.")

UnboundLocalError: local variable 'latitude' referenced before assignment

In [ ]:
session.close()

<hr style="border: 3px solid red;">

### Query database

In [ ]:
# Query all records and create a list with the returned data
query_result = session.query(RealState).all()
query_result[0:2]

In [ ]:
query_res = query_result[1]
print(f"house_id: {query_res.house_id}")
print(f"address: {query_res.address}")
print(f"price: {query_res.price}")
print(f"bed: {query_res.bed}")
print(f"bath: {query_res.bath}")
print(f"sqft: {query_res.sqft}")
print(f"lot: {query_res.lot}")
print(f"latitude: {query_res.latitude}")
print(f"longitude: {query_res.longitude}")
print(f"house_link: {query_res.house_link}")
print(f"image_1: {query_res.image_1}")
print(f"image_2: {query_res.image_2}")
print(f"map_link: {query_res.map_link}")
print(f"google_map: {query_res.google_map}")

<hr style="border: 3px solid red;">

### Basic Analysis

In [ ]:
# Query database using pandas
SQL_Query_df = pd.read_sql_query('''select * from realstatelisting''', engine)

In [ ]:
SQL_Query_df.describe()

In [ ]:
SQL_Query_df.head(2)

In [ ]:
ax1 = SQL_Query_df.plot.scatter(x='sqft',y='price',c='lot', colormap='viridis',figsize=(18,4))
ax1.set_xlabel("sqft", fontsize=14)
ax1.set_ylabel("price", fontsize=14)

In [ ]:
ax1 = SQL_Query_df['price'].plot(kind='hist',bins=15,figsize=(18,5));
ax1.set_xlabel("Price", fontsize=14);

In [ ]:
ax1 = SQL_Query_df.plot.scatter(x='latitude',y='longitude',c='price', colormap='viridis',figsize=(18,10))
ax1.set_xlabel("latitude", fontsize=14)
ax1.set_ylabel("longitude", fontsize=14)